In [1]:
import pandas as pd

In [2]:
df = pd.read_excel("../data/raw/estimaciones-y-proyecciones-2002-2035-comunas.xlsx").iloc[:-2]

# Cálculo de Personas por Estrato

Aquí se quiere calcular la cantidad de personas que hay en los siguientes estratos:

1. Hombres y Mujeres
2. Hombres
3. Mujeres
4. Población sobre 45 años
5. Población sobre 60 años
6. Población entre 60 a 70 años
7. Población recién nacidos

Para cada estrato se calculará la población que hay para el SSMO, Región Metropolitana y el País.

## 1. Hombres y Mujeres

In [3]:
COMUNAS_SSMO = [
    "Peñalolén",
    "Macul",
    "Ñuñoa",
    "La Reina",
    "Providencia",
    "Las Condes",
    "Vitacura",
    "Lo Barnechea",
]

poblacion_ssmo = df["Nombre Comuna"].isin(COMUNAS_SSMO)
suma_poblacion_ssmo = df[poblacion_ssmo].loc[:, "Poblacion 2017":"Poblacion 2035"].sum().reset_index()

poblacion_rm = (df["M"] == "Metropolitana de Santiago")
suma_poblacion_rm = df[poblacion_rm].loc[:, "Poblacion 2017":"Poblacion 2035"].sum().reset_index()

suma_poblacion_chile = df.loc[:, "Poblacion 2017":"Poblacion 2035"].sum().reset_index()

## 2. Hombres

In [4]:
poblacion_ssmo_hombre = (poblacion_ssmo) & (df["Sexo\n1=Hombre\n2=Mujer"] == 1.0)
suma_poblacion_ssmo_hombre = (
    df[poblacion_ssmo_hombre].loc[:, "Poblacion 2017":"Poblacion 2035"].sum().reset_index()
)

poblacion_rm_hombre = (poblacion_rm) & (df["Sexo\n1=Hombre\n2=Mujer"] == 1.0)
suma_poblacion_rm_hombre = (
    df[poblacion_rm_hombre].loc[:, "Poblacion 2017":"Poblacion 2035"].sum().reset_index()
)

poblacion_pais_hombre = df["Sexo\n1=Hombre\n2=Mujer"] == 1.0
suma_poblacion_chile_hombre = (
    df[poblacion_pais_hombre].loc[:, "Poblacion 2017":"Poblacion 2035"].sum().reset_index()
)

## 3. Mujeres

In [5]:
poblacion_ssmo_mujer = (poblacion_ssmo) & (df["Sexo\n1=Hombre\n2=Mujer"] == 2.0)
suma_poblacion_ssmo_mujer = (
    df[poblacion_ssmo_mujer].loc[:, "Poblacion 2017":"Poblacion 2035"].sum().reset_index()
)

poblacion_rm_mujer = (poblacion_rm) & (df["Sexo\n1=Hombre\n2=Mujer"] == 2.0)
suma_poblacion_rm_mujer = (
    df[poblacion_rm_mujer].loc[:, "Poblacion 2017":"Poblacion 2035"].sum().reset_index()
)

poblacion_pais_mujer = df["Sexo\n1=Hombre\n2=Mujer"] == 2.0
suma_poblacion_chile_mujer = (
    df[poblacion_pais_mujer].loc[:, "Poblacion 2017":"Poblacion 2035"].sum().reset_index()
)

## 4. Población sobre 45 años

In [6]:
poblacion_ssmo_sobre_45 = (poblacion_ssmo) & (df["Edad"] > 45)
suma_poblacion_ssmo_sobre_45 = (
    df[poblacion_ssmo_sobre_45].loc[:, "Poblacion 2017":"Poblacion 2035"].sum().reset_index()
)

poblacion_rm_sobre_45 = (poblacion_rm) & (df["Edad"] > 45)
suma_poblacion_rm_sobre_45 = (
    df[poblacion_rm_sobre_45].loc[:, "Poblacion 2017":"Poblacion 2035"].sum().reset_index()
)

poblacion_pais_sobre_45 = df["Edad"] > 45
suma_poblacion_chile_sobre_45 = (
    df[poblacion_pais_sobre_45].loc[:, "Poblacion 2017":"Poblacion 2035"].sum().reset_index()
)

## 5. Población sobre 60 años

In [7]:
poblacion_ssmo_sobre_60 = (poblacion_ssmo) & (df["Edad"] > 60)
suma_poblacion_ssmo_sobre_60 = (
    df[poblacion_ssmo_sobre_60].loc[:, "Poblacion 2017":"Poblacion 2035"].sum().reset_index()
)

poblacion_rm_sobre_60 = (poblacion_rm) & (df["Edad"] > 60)
suma_poblacion_rm_sobre_60 = (
    df[poblacion_rm_sobre_60].loc[:, "Poblacion 2017":"Poblacion 2035"].sum().reset_index()
)

poblacion_pais_sobre_60 = df["Edad"] > 60
suma_poblacion_chile_sobre_60 = (
    df[poblacion_pais_sobre_60].loc[:, "Poblacion 2017":"Poblacion 2035"].sum().reset_index()
)

## 6. Población entre 60 a 70 años

In [19]:
poblacion_ssmo_entre_60_a_70 = (poblacion_ssmo) & ((df["Edad"] >= 60) & (df["Edad"] <= 70))
suma_poblacion_ssmo_entre_60_a_70 = (
    df[poblacion_ssmo_entre_60_a_70].loc[:, "Poblacion 2017":"Poblacion 2035"].sum().reset_index()
)

poblacion_rm_entre_60_a_70 = (poblacion_rm) & ((df["Edad"] >= 60) & (df["Edad"] <= 70))
suma_poblacion_rm_entre_60_a_70 = (
    df[poblacion_rm_entre_60_a_70].loc[:, "Poblacion 2017":"Poblacion 2035"].sum().reset_index()
)

poblacion_pais_entre_60_a_70 = ((df["Edad"] >= 60) & (df["Edad"] <= 70))
suma_poblacion_chile_entre_60_a_70 = (
    df[poblacion_pais_entre_60_a_70].loc[:, "Poblacion 2017":"Poblacion 2035"].sum().reset_index()
)

## 7. Población Recién Nacidos Vivos

In [24]:
poblacion_ssmo_recien_nacidos = (poblacion_ssmo) & (df["Edad"] == 0)
suma_poblacion_ssmo_recien_nacidos = (
    df[poblacion_ssmo_recien_nacidos].loc[:, "Poblacion 2017":"Poblacion 2035"].sum().reset_index()
)

poblacion_rm_recien_nacidos = (poblacion_rm) & (df["Edad"] == 0)
suma_poblacion_rm_recien_nacidos = (
    df[poblacion_rm_recien_nacidos].loc[:, "Poblacion 2017":"Poblacion 2035"].sum().reset_index()
)

poblacion_pais_recien_nacidos = (df["Edad"] == 0)
suma_poblacion_chile_recien_nacidos = (
    df[poblacion_pais_recien_nacidos].loc[:, "Poblacion 2017":"Poblacion 2035"].sum().reset_index()
)